# Timescale Vector Store (PostgreSQL)

This notebook shows how to use the Postgres vector store `TimescaleVector` to store and query vector embeddings.

## What is Timescale Vector?
**[Timescale Vector](https://www.timescale.com/ai) is PostgreSQL++ for AI applications.**

Timescale Vector enables you to efficiently store and query billions of vector embeddings in `PostgreSQL`.
- Enhances `pgvector` with faster and more accurate similarity search on 1B+ vectors via DiskANN inspired indexing algorithm.
- Enables fast time-based vector search via automatic time-based partitioning and indexing.
- Provides a familiar SQL interface for querying vector embeddings and relational data.

Timescale Vector scales with you from POC to production:
- Simplifies operations by enabling you to store relational metadata, vector embeddings, and time-series data in a single database.
- Benefits from rock-solid PostgreSQL foundation with enterprise-grade feature liked streaming backups and replication, high-availability and row-level security.
- Enables a worry-free experience with enterprise-grade security and compliance.

## How to use Timescale Vector
Timescale Vector is available on [Timescale](https://www.timescale.com/products), the cloud PostgreSQL platform. (There is no self-hosted version at this time.)

- LlamaIndex users get a 90-day free trial for Timescale Vector.
- To get started, [signup](https://console.cloud.timescale.com/signup) to Timescale, create a new database and follow this notebook!
- See the [installation instructions](https://github.com/timescale/python-vector) for more details on using Timescale Vector in python.

## 0. Setup
Let's import everything we'll need for this notebook.

In [61]:
# import logging
# import sys

# Uncomment to see debug logs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import timescale_vector
from llama_index import SimpleDirectoryReader, StorageContext
from llama_index.indices.vector_store import VectorStoreIndex
from llama_index.vector_stores import TimescaleVectorStore
from llama_index.vector_stores.types import VectorStoreQuery, MetadataFilters
import textwrap
import openai

### Setup OpenAI API Key
To create embeddings for documents loaded into the index, let's configure your OpenAI API key:

In [62]:
# Get openAI api key by reading local .env file
# The .env file should contain a line starting with `OPENAI_API_KEY=sk-`
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# OR set it explicitly 
#import os
#os.environ["OPENAI_API_KEY"] = "<your key>"
openai.api_key = os.environ["OPENAI_API_KEY"]

### Create a PostgreSQL database and get a Timescale service URL
You need a service url to connect to your Timescale database instance.

To connect to your cloud PostgreSQL database in Timescale, you'll need your service URI, which can be found in the cheatsheet file you downloaded after creating a new database. The URI will look something like this: `postgres://tsdbadmin:<password>@<id>.tsdb.cloud.timescale.com:<port>/tsdb?sslmode=require`

In [64]:
# Get the service url by reading local .env file
# The .env file should contain a line starting with `TIMESCALE_SERVICE_URL=postgresql://`
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

TIMESCALE_SERVICE_URL = os.environ["TIMESCALE_SERVICE_URL"]

# OR set it explicitly
# TIMESCALE_SERVICE_URL = "postgres://tsdbadmin:<password>@<id>.tsdb.cloud.timescale.com:<port>/tsdb?sslmode=require"

## 1. Simple Similarity Search with Timescale Vector

### Loading documents
For this example, we'll use a [SimpleDirectoryReader](https://gpt-index.readthedocs.io/en/stable/examples/data_connectors/simple_directory_reader.html) to load the documents stored in the the `paul_graham_essay` directory. 

The `SimpleDirectoryReader` is one of LlamaIndex's most commonly used data connectors to read one or multiple files from a directory.

In [63]:
# load sample data from the data directory using a SimpleDirectoryReader
documents = SimpleDirectoryReader("../data/paul_graham").load_data()
print("Document ID:", documents[0].doc_id)

Document ID: 740ce1a1-4d95-40cc-b7f7-6d2874620a53


### Create a VectorStore Index with the TimescaleVectorStore
Next, to perform a similarity search, we first create a `TimescaleVector` [vector store](https://gpt-index.readthedocs.io/en/stable/core_modules/data_modules/storage/vector_stores.html) to store our vector embeddings from the essay content. TimescaleVectorStore takes a few arguments, namely the `service_url` which we loaded above, along with a `table_name` which we will be the name of the table that the vectors are stored in.

Then we create a [Vector Store Index](https://gpt-index.readthedocs.io/en/stable/community/integrations/vector_stores.html#vector-store-index) on the documents backed by Timescale using the previously documents.

In [65]:
# Create a TimescaleVectorStore to store the documents
vector_store = TimescaleVectorStore.from_params(
    service_url=TIMESCALE_SERVICE_URL,
    table_name="paul_graham_essay",
)

# Create a new VectorStoreIndex using the TimescaleVectorStore
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

### Query the index
Now that we've indexed the documents in our VectorStore, we can ask questions about our documents in the index by using the default `query_engine`.

Note you can also configure the query engine to configure the top_k most similar results returned, as well as metadata filters to filter the results by. See the [configure standard query setting section](https://gpt-index.readthedocs.io/en/stable/core_modules/data_modules/index/vector_store_guide.html) for more details.

In [66]:
query_engine = index.as_query_engine()
response = query_engine.query("Did the author work at YC?")

In [67]:
print(textwrap.fill(str(response), 100))

Yes, the author did work at YC.


In [68]:
response = query_engine.query("What did the author work on before college?")

In [69]:
print(textwrap.fill(str(response), 100))

Before college, the author worked on writing and programming. They wrote short stories and also
tried programming on the IBM 1401 computer using an early version of Fortran.


### Querying existing index
In the example above, we created a new Timescale Vector vectorstore and index from documents we loaded. Next we'll look at how to query an existing index. All we need is the service URI and the table name we want to access.

In [70]:
vector_store = TimescaleVectorStore.from_params(
    service_url=TIMESCALE_SERVICE_URL,
    table_name="paul_graham_essay",
)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do before YC?")

In [71]:
print(textwrap.fill(str(response), 100))

Before YC, the author wrote all of YC's internal software in Arc. They also worked on HN and had
three projects: writing essays, working on YC, and working in Arc. However, they gradually stopped
working on Arc due to time constraints and the increasing dependence on it for infrastructure.


## 2. Using ANN search indexes to speed up queries

(Note: These indexes are ANN indexes, and differ from the index concept in LlamaIndex)

You can speed up similarity queries by creating an index on the embedding column. You should only do this once you have ingested a large part of your data.

Timescale Vector supports the following indexes:
- timescale_vector_index: a disk-ann inspired graph index for fast similarity search (default).
- pgvector's HNSW index: a hierarchical navigable small world graph index for fast similarity search.
- pgvector's IVFFLAT index: an inverted file index for fast similarity search.

Important note: In PostgreSQL, each table can only have one index on a particular column. So if you'd like to test the performance of different index types, you can do so either by (1) creating multiple tables with different indexes, (2) creating multiple vector columns in the same table and creating different indexes on each column, or (3) by dropping and recreating the index on the same column and comparing results.

In [72]:
# Instantiate the TimescaleVectorStore from part 1
vector_store = TimescaleVectorStore.from_params(
    service_url=TIMESCALE_SERVICE_URL,
    table_name="paul_graham_essay",
)

Using the `create_index()` function without additional arguments will create a `timescale_vector (DiskANN)` index by default, using the default parameters.

In [73]:
# Create a timescale vector index (DiskANN)
vector_store.create_index()

You can also specify the parameters for the index. See the Timescale Vector documentation for a full discussion of the different parameters and their effects on performance.

In [74]:
#drop old index
vector_store.drop_index()

# create new timescale vector index (DiskANN) with specified parameters
vector_store.create_index("tsv", max_alpha=1.0, num_neighbors=50)

Timescale Vector also supports HNSW and ivfflat indexes:

In [75]:
vector_store.drop_index()

# Create an HNSW index
# Note: You don't need to specify m and ef_construction parameters as we set smart defaults.
vector_store.create_index("hnsw", m=16, ef_construction=64)

In [76]:
# Create an IVFFLAT index
# Note: You don't need to specify num_lists and num_records parameters as we set smart defaults.
vector_store.drop_index()
vector_store.create_index("ivfflat", num_lists=20, num_records=1000)

We recommend using `timescale-vector` or `HNSW` indexes in general.

In [78]:
# drop the ivfflat index
vector_store.drop_index()
# Create a timescale vector index (DiskANN)
vector_store.create_index()

## 3. Similarity Search with time-based filtering

A key use case for Timescale Vector is efficient time-based vector search. Timescale Vector enables this by automatically partitioning vectors (and associated metadata) by time. This allows you to efficiently query vectors by both similarity to a query vector and time.

Time-based vector search functionality is helpful for applications like:
- Storing and retrieving LLM response history (e.g. chatbots)
- Finding the most recent embeddings that are similar to a query vector (e.g recent news).
- Constraining similarity search to a relevant time range (e.g asking time-based questions about a knowledge base)

To illustrate how to use TimescaleVector's time-based vector search functionality, we'll use the git log history for TimescaleDB as a sample dataset and ask questions about it. Each git commit entry has a timestamp associated with it, as well as natural language message and other metadata (e.g author, commit hash etc). 

We'll illustrate how to create nodes with a time-based uuid and how run similarity searches with time range filters using the TimescaleVector vectorstore.

### Extract content and metadata from git log CSV file

First lets load in the git log csv file into a new collection in our PostgreSQL database named `timescale_commits`.

Note: Since this is a demo, we will only work with the first 1000 records. In practice, you can load as many records as you want.

In [79]:
import pandas as pd
from pathlib import Path
file_path = Path("../data/csv/commit_history.csv")
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Light data cleaning on CSV
df.dropna(inplace=True)
df = df.astype(str)
df = df[:1000]

In [80]:
# Take a look at the data in the csv (optional)
df.head()

,commit,author,date,change summary,change details
0,44e41c12ab25e36c202f58e068ced262eadc8d16,Lakshmi Narayanan Sreethar<lakshmi@timescale.com>,Tue Sep 5 21:03:21 2023 +0530,Fix segfault in set_integer_now_func,When an invalid function oid is passed to set_...
1,e66a40038e3c84fb1a68da67ad71caf75c64a027,Bharathy<satish.8483@gmail.com>,Sat Sep 2 09:24:31 2023 +0530,Fix server crash on UPDATE of compressed chunk,UPDATE query with system attributes in WHERE c...
2,c6a930897e9f9e9878db031cc7fb6ea79d721a74,Jan Nidzwetzki<jan@timescale.com>,Tue Aug 29 21:13:51 2023 +0200,Use Debian Bookworm for 32-bit tests,"So far, we have used Debian Buster (10) for ou..."
3,8e941b80ae1b0e0b6affe5431454cdc637628d99,Lakshmi Narayanan Sreethar<lakshmi@timescale.com>,Mon Aug 28 23:19:22 2023 +0530,Fix incorrect row count in EXPLAIN ANALYZE INS...,INSERT ... ON CONFLICT statements record few m...
4,caada43454e25d3098744fa6b675ac7d07390550,Lakshmi Narayanan Sreethar<lakshmi@timescale.com>,Tue May 30 20:32:29 2023 +0530,PG16: Fix concurrent update issues with MERGE.,PG16 commit postgres/postgres@9321c79c fixes a...


We'll define a helper funciton to create a uuid for a node and associated vector embedding based on its timestamp. We'll use this function to create a uuid for each git log entry.

Important note: If you are working with documents/nodes and want the current date and time associated with vector for time-based search, you can skip this step. A uuid will be automatically generated when the nodes are added to the table in Timescale Vector by default. In our case, because we want the uuid to be based on the timestamp in the past, we need to create the uuids manually.

In [81]:
from timescale_vector import client
# Function to take in a date string in the past and return a uuid v1
def create_uuid(date_string: str):
    if date_string is None:
        return None
    time_format = '%a %b %d %H:%M:%S %Y %z'
    datetime_obj = datetime.strptime(date_string, time_format)
    uuid = client.uuid_from_time(datetime_obj)
    return str(uuid)

In [82]:
# Helper functions
from typing import List, Tuple
# Helper function to split name and email given an author string consisting of Name Lastname <email>
def split_name(input_string: str) -> Tuple[str, str]:
    if input_string is None:
        return None, None
    start = input_string.find("<")
    end = input_string.find(">")
    name = input_string[:start].strip()
    return name

from datetime import datetime, timedelta
def create_date(input_string: str) -> datetime:
    if input_string is None:
        return None
    # Define a dictionary to map month abbreviations to their numerical equivalents
    month_dict = {
        "Jan": "01",
        "Feb": "02",
        "Mar": "03",
        "Apr": "04",
        "May": "05",
        "Jun": "06",
        "Jul": "07",
        "Aug": "08",
        "Sep": "09",
        "Oct": "10",
        "Nov": "11",
        "Dec": "12",
    }

    # Split the input string into its components
    components = input_string.split()
    # Extract relevant information
    day = components[2]
    month = month_dict[components[1]]
    year = components[4]
    time = components[3]
    timezone_offset_minutes = int(components[5])  # Convert the offset to minutes
    timezone_hours = timezone_offset_minutes // 60  # Calculate the hours
    timezone_minutes = timezone_offset_minutes % 60  # Calculate the remaining minutes
    # Create a formatted string for the timestamptz in PostgreSQL format
    timestamp_tz_str = f"{year}-{month}-{day} {time}+{timezone_hours:02}{timezone_minutes:02}"
    return timestamp_tz_str

Next, we'll define a function to create a `TextNode` for each git log entry. We'll use the helper function `create_uuid()` we defined above to create a uuid for each node based on its timestampe. And we'll use the helper functions `create_date()` and `split_name()` above to extract relevant metadata from the git log entry and add them to the node.

In [83]:
from llama_index.schema import TextNode, NodeRelationship, RelatedNodeInfo
# Create a Node object from a single row of data
def create_node(row):
    record = row.to_dict()
    record_name = split_name(record["author"])
    record_content = str(record["date"]) + " " + record_name + " " + str(record["change summary"]) + " " + str(record["change details"]) 
    # Can change to TextNode as needed
    node = TextNode(
        id_=create_uuid(record["date"]),
        text= record_content,
        metadata={
            'commit': record["commit"],
            'author': record_name,
            'date': create_date(record["date"]),
        }
    )
    return node

In [84]:
nodes = [create_node(row) for _, row in df.iterrows()]

Next we'll create vector embeddings of the content of each node so that we can perform similarity search on the text associated with each node. We'll use the `OpenAIEmbedding` model to create the embeddings.

In [85]:
# Create embeddings for nodes
from llama_index.embeddings import OpenAIEmbedding
embedding_model = OpenAIEmbedding()

for node in nodes:
    node_embedding = embedding_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

Let's examine the first node in our collection to see what it looks like.

In [87]:
print(nodes[0].get_content(metadata_mode="all"))

commit: 44e41c12ab25e36c202f58e068ced262eadc8d16
author: Lakshmi Narayanan Sreethar
date: 2023-09-5 21:03:21+0850

Tue Sep 5 21:03:21 2023 +0530 Lakshmi Narayanan Sreethar Fix segfault in set_integer_now_func When an invalid function oid is passed to set_integer_now_func, it finds out that the function oid is invalid but before throwing the error, it calls ReleaseSysCache on an invalid tuple causing a segfault. Fixed that by removing the invalid call to ReleaseSysCache.  Fixes #6037


In [88]:
print(nodes[0].get_embedding())

[-0.005366453900933266, 0.0016374519327655435, 0.005981510039418936, -0.026256779208779335, -0.03944991156458855, 0.026299940422177315, -0.0200558640062809, -0.01252412423491478, -0.04241368919610977, -0.004758591763675213, 0.05639812350273132, 0.006578581873327494, 0.014833281747996807, 0.009509989991784096, 0.0009675443288870156, -0.013157163746654987, -0.002265996066853404, -0.017048921436071396, 0.006553404498845339, -0.00217068032361567, 0.009085564874112606, 0.011775985360145569, -0.02514895796775818, -0.002679630182683468, 0.0030608929228037596, -3.439458305365406e-05, -0.00363818253390491, -0.03939236328005791, 0.0016806137282401323, -0.01207092497497797, 0.01739421673119068, -0.02241537719964981, -0.01753808930516243, -0.023782167583703995, -0.01598426327109337, -0.02575322426855564, -0.016876274719834328, -0.006380756851285696, -0.0009149408433586359, 0.00704616867005825, -0.0013290246715769172, -0.009776154533028603, -0.013200325891375542, -0.024832438677549362, -0.001940483

### Load documents and metadata into TimescaleVector vectorstore
Now that we have prepared our nodes and added embeddings to them, let's add them into our TimescaleVector vectorstore.

We'll create a Timescale Vector instance from the list of nodes we created.

First, we'll define a collection name, which will be the name of our table in the PostgreSQL database. 

We'll also define a time delta, which we pass to the `time_partition_interval` argument, which will be used to as the interval for partitioning the data by time. Each partition will consist of data for the specified length of time. We'll use 7 days for simplicity, but you can pick whatever value make sense for your use case -- for example if you query recent vectors frequently you might want to use a smaller time delta like 1 day, or if you query vectors over a decade long time period then you might want to use a larger time delta like 6 months or 1 year.

Then we'll add the nodes to the Timescale Vector vectorstore.

In [89]:
# Create a timescale vector store and add the newly created nodes to it
ts_vector_store = TimescaleVectorStore.from_params(
    service_url=TIMESCALE_SERVICE_URL,
    table_name="li_commit_history",
    time_partition_interval= timedelta(days=7),
)
ts_vector_store.add(nodes)

['8b407680-4c01-11ee-9cfb-34613327c1d4',
 '6bc8f580-4944-11ee-a5df-72c411556274',
 '300e0180-46a0-11ee-b1fa-5cbee2db2a5a',
 '38483d00-45cb-11ee-91ef-e28811f89485',
 'fee43080-fefa-11ed-b2c4-221d67576663',
 'faa8ea00-4686-11ee-b933-c2c7df407c25',
 'e9b9bf00-47d3-11ee-82f1-68496060cd6b',
 '4731fc80-466a-11ee-870c-0d447bf05033',
 '1b546b00-477f-11ee-a2f5-3b056a7b3b7e',
 '02a76f00-4705-11ee-bc0b-961fe977f891',
 '18138f00-465a-11ee-b474-0b4dd3cd7fbf',
 '4db77880-4658-11ee-9c87-c72f46f599bd',
 '01b10780-4649-11ee-a375-5719b2881af3',
 '89672980-45e9-11ee-aa6a-e6163fba3929',
 'ade6f400-4631-11ee-a763-1a12d02c3231',
 '6451c700-45a7-11ee-840b-38c386794716',
 '7dd61d00-456d-11ee-a1f1-16acb9b22a0f',
 '43e4d880-3770-11ee-9d73-0be6cef485f1',
 'ab6a8a00-44cb-11ee-9fbb-56d90e9b98f4',
 'ab6a8a00-44cb-11ee-932c-2ed39945682e',
 '3a835380-41e7-11ee-8536-65ae823be6a3',
 '6deb1f80-1b60-11ee-b371-e8d4ee149cc0',
 '1244e280-f570-11ed-af28-3a3923deff16',
 '5856c400-428c-11ee-82fa-b7e224c79026',
 '7a538d00-f15a-

### Querying vectors by time and similarity

Now that we have loaded our documents into TimescaleVector, we can query them by time and similarity.

TimescaleVector provides multiple methods for querying vectors by doing similarity search with time-based filtering Let's take a look at each method below.

First we define a query string and get the vector embedding for the query string.

In [90]:
# Define query and generate embedding for it
query_str = "What's new with TimescaleDB functions?"
embed_model = OpenAIEmbedding()
query_embedding = embed_model.get_query_embedding(query_str)

Then we set some variables which we'll use in our time filters.

In [91]:
# Time filter variables for query
start_dt = datetime(2023, 8, 1, 22, 10, 35) # Start date = 1 August 2023, 22:10:35
end_dt = datetime(2023, 8, 30, 22, 10, 35) # End date = 30 August 2023, 22:10:35
td = timedelta(days=7) # Time delta = 7 days

Method 1: Filter within a provided start date and end date.

In [93]:
# Query the vector database
vector_store_query = VectorStoreQuery(query_embedding = query_embedding, similarity_top_k=5)

# return most similar vectors to query between start date and end date date range
# returns a VectorStoreQueryResult object 
query_result = ts_vector_store.query(vector_store_query, start_date = start_dt, end_date = end_dt)
query_result

VectorStoreQueryResult(nodes=[TextNode(id_='22747180-31f1-11ee-bd8e-101e36c28c91', embedding=None, metadata={'commit': ' 7aeed663b9c0f337b530fd6cad47704a51a9b2ec', 'author': 'Dmitry Simonenko', 'date': '2023-08-3 14:30:23+0500'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='3273f20a98f02c75847896b929888b05e8751ae5e258d7feb8605bd5290ef8ca', text='Thu Aug 3 14:30:23 2023 +0300 Dmitry Simonenko Feature flags for TimescaleDB features This PR adds several GUCs which allow to enable/disable major timescaledb features:  - enable_hypertable_create - enable_hypertable_compression - enable_cagg_create - enable_policy_create ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), TextNode(id_='faa8ea00-4686-11ee-b933-c2c7df407c25', embedding=None, metadata={'commit': ' e4facda540286b0affba47ccc63959fefe2a7b26', 'author': 'Sven Klemm', 'date': '2023-08-29 18:13:2

Let's inspect the nodes that were returned from the similarity search:

In [94]:
# for each node in the query result, print the node metadata date
for node in query_result.nodes:
    print("-" * 80)
    print(node.metadata["date"])
    print(node.get_content(metadata_mode="all"))

--------------------------------------------------------------------------------
2023-08-3 14:30:23+0500
commit:  7aeed663b9c0f337b530fd6cad47704a51a9b2ec
author: Dmitry Simonenko
date: 2023-08-3 14:30:23+0500

Thu Aug 3 14:30:23 2023 +0300 Dmitry Simonenko Feature flags for TimescaleDB features This PR adds several GUCs which allow to enable/disable major timescaledb features:  - enable_hypertable_create - enable_hypertable_compression - enable_cagg_create - enable_policy_create
--------------------------------------------------------------------------------
2023-08-29 18:13:24+0320
commit:  e4facda540286b0affba47ccc63959fefe2a7b26
author: Sven Klemm
date: 2023-08-29 18:13:24+0320

Tue Aug 29 18:13:24 2023 +0200 Sven Klemm Add compatibility layer for _timescaledb_internal functions With timescaledb 2.12 all the functions present in _timescaledb_internal were moved into the _timescaledb_functions schema to improve schema security. This patch adds a compatibility layer so external calle

Note how the query only returns results within the specified date range.

Method 2: Filter within a provided start date, and a time delta later.


In [95]:
vector_store_query = VectorStoreQuery(query_embedding = query_embedding, similarity_top_k=5)

# return most similar vectors to query from start date and a time delta later
query_result = ts_vector_store.query(vector_store_query, start_date = start_dt, time_delta = td)

for node in query_result.nodes:
    print("-" * 80)
    print(node.metadata["date"])
    print(node.get_content(metadata_mode="all"))

--------------------------------------------------------------------------------
2023-08-3 14:30:23+0500
commit:  7aeed663b9c0f337b530fd6cad47704a51a9b2ec
author: Dmitry Simonenko
date: 2023-08-3 14:30:23+0500

Thu Aug 3 14:30:23 2023 +0300 Dmitry Simonenko Feature flags for TimescaleDB features This PR adds several GUCs which allow to enable/disable major timescaledb features:  - enable_hypertable_create - enable_hypertable_compression - enable_cagg_create - enable_policy_create
--------------------------------------------------------------------------------
2023-08-7 19:49:47+-500
commit:  5bba74a2ec083728f8e93e09d03d102568fd72b5
author: Fabrízio de Royes Mello
date: 2023-08-7 19:49:47+-500

Mon Aug 7 19:49:47 2023 -0300 Fabrízio de Royes Mello Relax strong table lock when refreshing a CAGG When refreshing a Continuous Aggregate we take a table lock on _timescaledb_catalog.continuous_aggs_invalidation_threshold when processing the invalidation logs (the first transaction of the refre

Once again, notice how only nodes between the start date (1 August) and the defined time delta later (7 days later) are returned.

Method 3: Filter within a provided end date and a time delta earlier.

In [96]:
vector_store_query = VectorStoreQuery(query_embedding = query_embedding, similarity_top_k=5)

# return most similar vectors to query from end date and a time delta earlier
query_result = ts_vector_store.query(vector_store_query, end_date = end_dt, time_delta = td)

for node in query_result.nodes:
    print("-" * 80)
    print(node.metadata["date"])
    print(node.get_content(metadata_mode="all"))

--------------------------------------------------------------------------------
2023-08-29 18:13:24+0320
commit:  e4facda540286b0affba47ccc63959fefe2a7b26
author: Sven Klemm
date: 2023-08-29 18:13:24+0320

Tue Aug 29 18:13:24 2023 +0200 Sven Klemm Add compatibility layer for _timescaledb_internal functions With timescaledb 2.12 all the functions present in _timescaledb_internal were moved into the _timescaledb_functions schema to improve schema security. This patch adds a compatibility layer so external callers of these internal functions will not break and allow for more flexibility when migrating.
--------------------------------------------------------------------------------
2023-08-29 10:49:47+0320
commit:  a9751ccd5eb030026d7b975d22753f5964972389
author: Sven Klemm
date: 2023-08-29 10:49:47+0320

Tue Aug 29 10:49:47 2023 +0200 Sven Klemm Move partitioning functions to _timescaledb_functions schema To increase schema security we do not want to mix our own internal objects with us

The main takeaway is that in each result above, only vectors within the specified time range are returned. These queries are very efficient as they only need to search the relevant partitions.

## 4. Using TimescaleVector store as a Retriever and Query engine 

Now that we've explored basic similarity search and similarity search with time-based filters, let's look at how to these features of Timescale Vector with LLamaIndex's retriever and query engine.

First we'll look at how to use TimescaleVector as a [retriever](https://gpt-index.readthedocs.io/en/latest/api_reference/query/retrievers.html), specifically a [Vector Store Retriever](https://gpt-index.readthedocs.io/en/latest/api_reference/query/retrievers/vector_store.html).

To constrain the nodes retrieved to a relevant time-range, we can use TimescaleVector's time filters. We simply pass the time filter parameters as `vector_strored_kwargs` when creating the retriever.

In [97]:
from llama_index import VectorStoreIndex
from llama_index.storage import StorageContext

index = VectorStoreIndex.from_vector_store(ts_vector_store)
retriever = index.as_retriever(vector_store_kwargs = ({"start_date": start_dt, "time_delta":td}))
retriever.retrieve("What's new with TimescaleDB functions?")

[NodeWithScore(node=TextNode(id_='22747180-31f1-11ee-bd8e-101e36c28c91', embedding=None, metadata={'commit': ' 7aeed663b9c0f337b530fd6cad47704a51a9b2ec', 'author': 'Dmitry Simonenko', 'date': '2023-08-3 14:30:23+0500'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='3273f20a98f02c75847896b929888b05e8751ae5e258d7feb8605bd5290ef8ca', text='Thu Aug 3 14:30:23 2023 +0300 Dmitry Simonenko Feature flags for TimescaleDB features This PR adds several GUCs which allow to enable/disable major timescaledb features:  - enable_hypertable_create - enable_hypertable_compression - enable_cagg_create - enable_policy_create ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.1813839050209377),
 NodeWithScore(node=TextNode(id_='b5583780-3574-11ee-871a-5a8c45d660c8', embedding=None, metadata={'commit': ' 5bba74a2ec083728f8e93e09d03d102568fd72b5', 'author': 'Fab

Next we'll look at how to use TimescaleVector as a [query engine](https://gpt-index.readthedocs.io/en/latest/api_reference/query/query_engines.html).

Once again, we use TimescaleVector's time filters to constrain the search to a relevant time range by passing our time filter parameters as `vector_strored_kwargs` when creating the query engine.

In [99]:
index = VectorStoreIndex.from_vector_store(ts_vector_store)
query_engine = index.as_query_engine(vector_store_kwargs = ({"start_date": start_dt, "end_date":end_dt}))

#query_str = "What's new with TimescaleDB? List 3 new features"
query_str = "What's new with TimescaleDB functions? When were these changes made and by whom?"
response = query_engine.query(query_str)
print(str(response))

TimescaleDB functions have undergone changes recently. These changes were made by Sven Klemm on August 29, 2023. The changes involve adding a compatibility layer for _timescaledb_internal functions. This layer ensures that external callers of these internal functions will not break and allows for more flexibility when migrating.
